In [1]:
from collections import defaultdict as dd
import os
import json
import sqlite3
import ast
import re
from sqlite3 import Error
from collections import Counter

import sys
sys.path.append("rat-sql")
from ratsql.datasets.spider_lib.process_sql import get_schema, Schema, get_sql
from ratsql.datasets.spider_lib.evaluation import Evaluator

from ratsql import datasets
from ratsql.utils import registry
from ratsql.datasets.spider_lib import evaluation

In [2]:
config_temp = {
            "db_path": "rat-sql/data/spider/database",
            "name": "spider",
            "paths": [
                "rat-sql/data/spider/train_spider.json",
                "rat-sql/data/spider/train_others.json"
            ],
            "tables_paths": [
                "rat-sql/data/spider/tables.json"
            ]
        }

data = registry.construct('dataset', config_temp)

spider_db_loc = 'rat-sql/data/spider/database/'

ratsql_infereval_loc = 'rat-sql/logdir/bert_run/bs=12,lr=7.4e-04,bert_lr=3.0e-06,end_lr=0e0,att=1/ie_dirs/'
ratsql_train = 'rat-sql/data/spider/train_spider.json'
ratsql_train_others = 'rat-sql/data/spider/train_others.json'
step = 
ratsql_infer = os.path.join(ratsql_infereval_loc, f'bert_run_true_1-step{step}.infer')
ratsql_eval  = os.path.join(ratsql_infereval_loc, f'bert_run_true_1-step{step}.eval')

DB connections: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 166/166 [00:00<00:00, 271.07it/s]


In [49]:
db_path = os.path.join(spider_db_loc, 'department_management/department_management.sqlite')
schema = Schema(get_schema(db_path))
gold = 'SELECT count(*) FROM head WHERE age  >  56'

g_sql = get_sql(schema, gold)

foreign_key_maps = {
    db_id: evaluation.build_foreign_key_map(schema.orig)
    for db_id, schema in data.schemas.items()
}
evaluator = evaluation.Evaluator(
    data.db_path,
    foreign_key_maps,
    'match')

evaluator.eval_hardness(g_sql) 


In [4]:
def read_json(file):
    with open(file) as json_file:
        json_data = json.load(json_file)
    
    return json_data

def read_jsonl(file):
    with open(file) as json_file:
        json_data = [json.loads(line) for line in json_file]
    
    return json_data

def desc_by_hardness(total_scores, key):
    return round(total_scores[key], 2)

def get_table_desc(db):
    schema = {}
    conn = sqlite3.connect(db)
    conn.text_factory = str
    cursor = conn.cursor()

    # fetch table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [str(table[0].lower()) for table in cursor.fetchall()]

    for table in tables:
        schema[table] = {'columns': [], 'col_length': 0, 'row_length': 0}
        cursor.execute("PRAGMA table_info({})".format(table))
        schema[table]['columns'] = [str(col[1].lower()) for col in cursor.fetchall()]

        cursor.execute(f"select * from {table}")

        schema[table]['col_length'] = len(schema[table]['columns'])
        schema[table]['row_length'] = len(cursor.fetchall())

    return schema

def gold_table_cnt(item):
    gold = item['gold']

    p = re.compile('T[0-9]')
    used_table = set(list(p.findall(gold)))

    print(gold)
    print(used_table)
    print(len(used_table) if used_table else 0)
    
    return used_table

In [5]:
train_dataset = read_json(ratsql_train)
train_others = read_json(ratsql_train_others)

train_all = train_dataset + train_others
print(len(train_all))

db_length = len(os.listdir('rat-sql/data/spider/database/'))
print(f'How many DB: {db_length}')

8659
How many DB: 166


In [ ]:
train_hardness = []
failed = 0

for data in train_all:
    try:
        db_id = data['db_id']
        db_path = os.path.join(spider_db_loc, f'{db_id}/{db_id}.sqlite')
        schema = Schema(get_schema(db_path))
        gold = data['query']

        g_sql = get_sql(schema, gold)

        train_hardness.append(evaluator.eval_hardness(g_sql))
    except Exception as e:
        print(e)
        failed += 1

train_hardness_dic = Counter(train_hardness)

In [7]:
infer = read_jsonl(ratsql_infer)
eval  = read_jsonl(ratsql_eval)[0]

eval_peritem = eval['per_item']
eval_total_scores = eval['total_scores']

FileNotFoundError: [Errno 2] No such file or directory: 'rat-sql/logdir/bert_run/bs=12,lr=7.4e-04,bert_lr=3.0e-06,end_lr=0e0,att=1/ie_dirs/bert_run_true_1-step30100.infer'

In [6]:
print(f"Easy Accuracy: {desc_by_hardness(eval_total_scores, 'easy')['exact']}")
print(f"Medium Accuracy: {desc_by_hardness(eval_total_scores, 'medium')['exact']}")
print(f"Hard Accuracy: {desc_by_hardness(eval_total_scores, 'hard')['exact']}")
print(f"Extra Accuracy: {desc_by_hardness(eval_total_scores, 'extra')['exact']}")

Easy Accuracy: 0.7741935483870968
Medium Accuracy: 0.547085201793722
Hard Accuracy: 0.5114942528735632
Extra Accuracy: 0.3433734939759036


In [6]:
dev_hardness_dic = {'easy':0, 'medium':0, 'hard':0, 'extra':0}

for i in ['easy', 'medium', 'hard', 'extra']:
    cnt = desc_by_hardness(eval_total_scores, i)['count']
    dev_hardness_dic[i] += cnt

NameError: name 'eval_total_scores' is not defined

In [76]:
train_hardness_dic = Counter(train_hardness)

for key in dev_hardness_dic:
    train_hardness_dic[key] += dev_hardness_dic[key]

train_hardness_dic

Counter({'easy': 2231, 'medium': 3445, 'hard': 2095, 'extra': 1921})

In [ ]:
infer = read_jsonl(ratsql_infer)
eval  = read_jsonl(ratsql_eval)[0]

eval_peritem = eval['per_item']
eval_total_scores = eval['total_scores']

In [78]:
exact_true = []
exact_false = []

for i in eval_peritem:
    if i['exact']:
        exact_true.append(i)
    else:
        exact_false.append(i)

In [167]:
from collections import defaultdict as dd

dict = dd(lambda: {})

In [163]:
from collections import defaultdict as dd

level = {'easy': 0, 'medium': 0, 'hard': 0, 'extra': 0}
cnt = {'easy':0, 'medium': 0, 'hard': 0, 'extra': 0}
for i in eval_peritem:
    rule = re.compile('T[1-9]')
    found = len(set(rule.findall(i['gold'])))
    if found:
        level[i['hardness']] += found
    else:
        level[i['hardness']] += 1
    cnt[i['hardness']] += 1


In [172]:
print(f'Level cnt: {level}')
print(f'Table Count: {cnt}')
for key in cnt:
    print(f'Table Mean: {level[key] / cnt[key]}')

Level cnt: {'easy': 254, 'medium': 629, 'hard': 263, 'extra': 335}
Table Count: {'easy': 248, 'medium': 446, 'hard': 174, 'extra': 166}
Table Mean: 1.0241935483870968
Table Mean: 1.4103139013452914
Table Mean: 1.5114942528735633
Table Mean: 2.0180722891566263


In [81]:
correct_easy, correct_medium, correct_hard, correct_extra = 0,0,0,0

for i in exact_true:
    if i['hardness'] == 'easy':
        correct_easy += 1
    elif i['hardness'] == 'medium':
        correct_medium += 1
    elif i['hardness'] == 'hard':
        correct_hard += 1
    else:
        correct_extra += 1

print(correct_easy, correct_medium, correct_hard, correct_extra)

192 244 89 57


In [83]:
wrong_easy, wrong_medium, wrong_hard, wrong_extra = 0,0,0,0

for i in exact_false:
    if i['hardness'] == 'easy':
        wrong_easy += 1
    elif i['hardness'] == 'medium':
        wrong_medium += 1
    elif i['hardness'] == 'hard':
        wrong_hard += 1
    else:
        wrong_extra += 1

print(wrong_easy, wrong_medium, wrong_hard, wrong_extra)

56 202 85 109


In [121]:
wrong_easy, wrong_medium, wrong_hard, wrong_extra = [],[],[],[]
error = 0

for wrong in exact_false:
    db_name = wrong['db_name']
    db_path = os.path.join(spider_db_loc, f'{db_name}/{db_name}.sqlite')
    hardness = wrong['hardness']

    try:
        desc = get_table_desc(db_path)

        if hardness == 'easy' and desc not in wrong_easy:
            wrong_easy.append(desc)
        elif hardness == 'medium' and desc not in wrong_medium:
            wrong_medium.append(desc)
        elif hardness == 'hard' and desc not in wrong_hard:
            wrong_hard.append(desc)
        elif hardness == 'extra' and desc not in wrong_extra:
            wrong_extra.append(desc)
    except Exception as e:
        # print(e)
        error += 1

In [271]:
# Table Description

db_name = 'cre_Doc_Template_Mgt'
db_path = os.path.join(spider_db_loc, f'{db_name}/{db_name}.sqlite')

get_table_desc(db_path)

{'ref_template_types': {'columns': ['template_type_code',
   'template_type_description'],
  'col_length': 2,
  'row_length': 5},
 'templates': {'columns': ['template_id',
   'version_number',
   'template_type_code',
   'date_effective_from',
   'date_effective_to',
   'template_details'],
  'col_length': 6,
  'row_length': 20},
 'documents': {'columns': ['document_id',
   'template_id',
   'document_name',
   'document_description',
   'other_details'],
  'col_length': 5,
  'row_length': 15},
 'paragraphs': {'columns': ['paragraph_id',
   'document_id',
   'paragraph_text',
   'other_details'],
  'col_length': 4,
  'row_length': 15}}

In [174]:
# 난이도 별로 어떤 Clause에서 오류를 범하는가?
wrong_easy, wrong_medium, wrong_hard, wrong_extra = [],[],[],[]
error = 0

for wrong in exact_false:
    db_name = wrong['db_name']
    db_path = os.path.join(spider_db_loc, f'{db_name}/{db_name}.sqlite')
    hardness = wrong['hardness']

    try:
        desc = get_table_desc(db_path)

        if hardness == 'easy':
            wrong_easy.append(desc)
        elif hardness == 'medium':
            wrong_medium.append(desc)
        elif hardness == 'hard':
            wrong_hard.append(desc)
        elif hardness == 'extra':
            wrong_extra.append(desc)
    except Exception as e:
        # print(e)
        error += 1

[{'stadium': {'columns': ['stadium_id',
    'location',
    'name',
    'capacity',
    'highest',
    'lowest',
    'average'],
   'col_length': 7,
   'row_length': 9},
  'singer': {'columns': ['singer_id',
    'name',
    'country',
    'song_name',
    'song_release_year',
    'age',
    'is_male'],
   'col_length': 7,
   'row_length': 6},
  'concert': {'columns': ['concert_id',
    'concert_name',
    'theme',
    'stadium_id',
    'year'],
   'col_length': 5,
   'row_length': 6},
  'singer_in_concert': {'columns': ['concert_id', 'singer_id'],
   'col_length': 2,
   'row_length': 10}},
 {'student': {'columns': ['stuid',
    'lname',
    'fname',
    'age',
    'sex',
    'major',
    'advisor',
    'city_code'],
   'col_length': 8,
   'row_length': 34},
  'has_pet': {'columns': ['stuid', 'petid'], 'col_length': 2, 'row_length': 3},
  'pets': {'columns': ['petid', 'pettype', 'pet_age', 'weight'],
   'col_length': 4,
   'row_length': 3}},
 {'continents': {'columns': ['contid', 'conti

In [127]:
partial_keys = exact_false[0]['partial'].keys()

In [176]:
partial_dic = {'easy':{}, 'medium': {}, 'hard':{}, 'extra':{}}
for kk in partial_dic:
    for k in partial_keys:
        partial_dic[kk][k] = 0

partial_dict = {'easy':{}, 'medium': {}, 'hard':{}, 'extra':{}}
for kk in partial_dict:
    for k in partial_keys:
        partial_dict[kk][k] = 0

# Partial Accuracy

for data in exact_false:
    hardness = data['hardness']
    
    for part_key in data['partial']:
        print(data['partial'][part_key]['acc'])
        if data['partial'][part_key]['acc']:
            partial_dic[hardness][part_key] += 1
        elif not data['partial'][part_key]['acc']:
            partial_dict[hardness][part_key] += 1

partial_dict

In [ ]:
db_path = os.path.join(spider_db_loc, '')

In [149]:
train_dev_tables =read_json('rat-sql/data/spider/tables.json')

Table 개수가 성능에 미치는 영향

In [185]:
get_table_desc(os.path.join(spider_db_loc, 'flight_1/flight_1.sqlite'))

{'flight': {'columns': ['flno',
   'origin',
   'destination',
   'distance',
   'departure_date',
   'arrival_date',
   'price',
   'aid'],
  'col_length': 8,
  'row_length': 10},
 'aircraft': {'columns': ['aid', 'name', 'distance'],
  'col_length': 3,
  'row_length': 16},
 'employee': {'columns': ['eid', 'name', 'salary'],
  'col_length': 3,
  'row_length': 31},
 'certificate': {'columns': ['eid', 'aid'], 'col_length': 2, 'row_length': 69}}

난이도 별 Question / SQL Length

In [194]:
infer_data = read_jsonl(ratsql_infer)

In [278]:
for i in infer_data:
    print(i['beams'][0]['orig_question'])

How many singers do we have?
What is the total number of singers?
Show name, country, age for all singers ordered by age from the oldest to the youngest.
What are the names, countries, and ages for every singer in descending order of age?
What is the average, minimum, and maximum age of all singers from France?
What is the average, minimum, and maximum age for all French singers?
Show the name and the release year of the song by the youngest singer.
What are the names and release years for all the songs of the youngest singer?
What are all distinct countries where singers above age 20 are from?
What are  the different countries with singers above age 20?
Show all countries and the number of singers in each country.
How many singers are from each country?
List all song names by singers above the average age.
What are all the song names by singers who are older than average?
Show location and name for all stadiums with a capacity between 5000 and 10000.
What are the locations and names o

In [265]:
def length(items):
    level = {'easy': 0, 'medium': 0, 'hard': 0, 'extra': 0}
    cnt = {'easy':0, 'medium': 0, 'hard': 0, 'extra': 0}  
    mean = {'easy':0, 'medium': 0, 'hard': 0, 'extra': 0}  

    s_level = {'easy': 0, 'medium': 0, 'hard': 0, 'extra': 0}
    s_cnt = {'easy':0, 'medium': 0, 'hard': 0, 'extra': 0}  
    s_mean = {'easy':0, 'medium': 0, 'hard': 0, 'extra': 0}  

    for data in items:
        try:
            question_length = len(data['question_toks'])
            sql = data['query_toks']
            rule = re.compile('T[1-9]')
            sub_sql = [re.sub(rule, '', s) for s in sql]
            
            sql_length = len([i for i in sql if i.isupper()])

            db_id = data['db_id']
            db_path = os.path.join(spider_db_loc, f'{db_id}/{db_id}.sqlite')
            schema = Schema(get_schema(db_path))
            gold = data['query']

            g_sql = get_sql(schema, gold)

            hardness = evaluator.eval_hardness(g_sql)

            level[hardness] += question_length
            cnt[hardness] += 1  

            mean[hardness] = level[hardness] / cnt[hardness]

            s_level[hardness] += sql_length
            s_cnt[hardness] += 1  

            s_mean[hardness] = s_level[hardness] / s_cnt[hardness]
 
        except Exception as e:
            print(e)
            pass

    return level, cnt, mean, s_level, s_cnt, s_mean


In [266]:
level, cnt, mean, s_level, s_cnt, s_mean = length(train_all)

'ref_company_types'


## Hardness 기준으로 Dataset 정리

In [272]:
_easy, _medium, _hard, _extra = [],[],[],[]


for item in eval_peritem:
    db_name = item['db_name']
    db_path = os.path.join(spider_db_loc, f'{db_name}/{db_name}.sqlite')
    hardness = item['hardness']

    try:
        desc = get_table_desc(db_path)

        if hardness == 'easy':
            _easy.append(desc)
        elif hardness == 'medium':
            _medium.append(desc)
        elif hardness == 'hard':
            _hard.append(desc)
        elif hardness == 'extra':
            _extra.append(desc)
    except Exception as e:
        error += 1